# 라이브러리 Install

In [17]:
%pip install --upgrade pip

%pip install qdrant_client rank_bm25 sentence_transformers

  Using cached pip-25.3-py3-none-any.whl.metadata (4.7 kB)
Using cached pip-25.3-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 25.1.1
    Uninstalling pip-25.1.1:
      Successfully uninstalled pip-25.1.1
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# 라이브러리 Import

In [25]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, SparseVectorParams, PointStruct, SparseVector
from sentence_transformers import SentenceTransformer
from rank_bm25 import BM25Okapi

import numpy as np

# 임베딩 모델, 벡터 DB 초기화

  📊 사용 가능한 로컬 임베딩 모델

  | 모델                                    | 차원  | 속도     | 품질      | 용도                    |
  |---------------------------------------|-----|--------|---------|-----------------------|
  | all-MiniLM-L6-v2                      | 384 | ⚡⚡⚡ 빠름 | ⭐⭐⭐ 좋음  | 빠른 테스트                |
  | all-mpnet-base-v2                     | 768 | ⚡⚡ 중간  | ⭐⭐⭐⭐ 우수 | 추천 (Vertex AI와 동일 차원) |
  | paraphrase-multilingual-mpnet-base-v2 | 768 | ⚡⚡ 중간  | ⭐⭐⭐⭐ 우수 | 한국어 지원 좋음             |

In [ ]:
# ========================================
# 1. 초기화
# ========================================
model = "all-MiniLM-L6-v2"
sentenceTransformer = SentenceTransformer(model)
print(f"✅ SentenceTransformer 모델 초기화 완료 ({model})")

qdrantClient = QdrantClient(":memory:")
print("✅ Qdrant 생성 완료 (메모리 모드)")

✅ SentenceTransformer 모델 초기화 완료 (all-MiniLM-L6-v2)
✅ Qdrant 생성 완료 (메모리 모드)


# 컬렉션 생성

In [ ]:
# ========================================
# 2. 컬렉션 생성
# ========================================
collection_name="test"

if qdrantClient.collection_exists(collection_name=collection_name):
    qdrantClient.delete_collection(collection_name=collection_name)
    print("✅ Qdrant 기존 컬렉션 삭제 완료")

qdrantClient.create_collection(
    collection_name=collection_name,
    vectors_config={
        "dense": VectorParams(size=384, distance=Distance.COSINE)
    },
    sparse_vectors_config={
        "sparse": SparseVectorParams()
    }
)
print("✅ Qdrant 컬렉션 생성 완료")

✅ Qdrant 기존 컬렉션 삭제 완료
✅ Qdrant 컬렉션 생성 완료


# 테스트 데이터 생성

In [ ]:
# ========================================
# 3. 테스트 데이터 삽입
# =======================================
docs = [
    "미쉐린 3스타 프렌치 파인다이닝, 계절 식재료를 활용한 모던 퀴진",
    "블루리본 이탈리안 레스토랑, 홈메이드 파스타와 리조또 전문",
    "강남 웨이팅 맛집, SNS에서 화제인 인기 브런치 카페",
    "전통 일식 오마카세, 제철 생선으로 만드는 스시 코스",
    "한남동 바베큐 레스토랑, 프리미엄 숙성육 전문점"
]

# 임베딩 & BM25

In [ ]:
# Dense 임베딩
dense_embeddings = sentenceTransformer.encode(docs)
print(f"✅ Dense 임베딩 생성: {dense_embeddings.shape}")

# BM25 인덱스 (전체 문서로 학습)
bm25 = BM25Okapi([doc.split() for doc in docs])
print(f"✅ BM25 인덱싱 완료")

# 저장

In [ ]:
# Qdrant에 삽입
points = []
for idx, (doc, dense) in enumerate(zip(docs, dense_embeddings)):
    sparse_scores = bm25.get_scores(doc.split())
    sparse_idx = np.where(sparse_scores > 0)[0]

    points.append(PointStruct(
        id=idx,
        vector={
            "dense": dense.tolist(),
            "sparse": SparseVector(
                indices=sparse_idx.tolist(),
                values=sparse_scores[sparse_idx].tolist(),
            )
        },
        payload={"text": doc}
    ))

qdrantClient.upsert(collection_name=collection_name, points=points)
print(f"✅ {len(points)}개 포인트 삽입 완료")

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

# Hybrid 검색 (Dense + Sparse)

In [ ]:
query = "프렌치 레스토랑"

# query -> Dense 임베딩 생성
query_dense_embedding = sentenceTransformer.encode(query)

# query -> Dense 검색
dense_results = qdrantClient.query_points(
    collection_name=collection_name,
    query=query_dense_embedding.tolist(),
    using="dense",
    limit=10
)

# Sparse 쿼리 벡터 (BM25)
query_tokens = query.split()
query_sparse_scores = bm25.get_scores(query_tokens)
query_sparse_idx = np.where(query_sparse_scores > 0)[0]

# Hybrid 검색 (Qdrant v1.7+)
results = qdrantClient.query_points(
    collection_name=collection_name,
    prefetch=[
        {
            # 1단계: Sparse로 후보 검색
            "query": SparseVector(
                indices=query_sparse_idx.tolist(),
                values=query_sparse_scores[query_sparse_idx].tolist()
            ),
            "using": "sparse",
            "limit": 10
        }
    ],
    # 2단계: Dense로 재랭킹
    
)